In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib.request

import numpy as np
import tensorflow as tf
import base64

# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"


# If the training and test sets aren't stored locally, download them.
if not os.path.exists(IRIS_TRAINING):
    raw = urllib.request.urlopen(IRIS_TRAINING_URL).read().decode('utf-8')
    with open(IRIS_TRAINING, "w") as f:
      f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urllib.request.urlopen(IRIS_TEST_URL).read().decode('utf-8')
    with open(IRIS_TEST, "w") as f:
      f.write(raw)

# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
  filename=IRIS_TRAINING,
  target_dtype=np.int,
  features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
  filename=IRIS_TEST,
  target_dtype=np.int,
  features_dtype=np.float32)

# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

Instructions for updating:
Use tf.data instead.


In [51]:
feature_columns

[_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None)]

In [28]:
# Build 3 layer DNN with 10, 20, 30 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns = feature_columns,
                                           hidden_units = [10, 20, 30],
                                           n_classes = 3,
                                           model_dir = 'tmp/iris_model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C8FBF91208>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'tmp/iris_model'}


In [30]:
def get_input_train():
    X = tf.constant(training_set.data)
    y = tf.constant(training_set.target)
    return X, y

def get_input_test():
    X = tf.constant(test_set.data)
    y = tf.constant(test_set.target)
    return X, y

In [32]:
# Fit model.
classifier.fit(input_fn = get_input_train, steps=2000)

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into tmp/iris_model\model.ckpt.
INFO:tensorflow:loss = 1.1535661, step = 1
INFO:tensorflow:global_step/sec: 135.446
INFO:tensorflow:loss = 0.10913229, step = 101 (0.723 sec)
INFO:tensorflow:global_step/sec: 213.365
INFO:tensorflow:loss = 0.07016535, step = 201 (0.484 sec)
INFO:tensorflow:global_step/sec: 206.487
INFO:tensorflow:loss = 0.061482508, step = 301 (0.484 sec)
INFO:tensorflow:global_step/sec: 200.035
INFO:tensorflow:loss = 0.05725072, step = 401 (0.484 sec)
INFO:tensorflow:global_step/sec: 206.489
INFO:tensorflow:loss = 0.054212973, step = 501 (0.500 sec)
INFO:tensorflow:global_step/sec: 206.488
INFO:tensorf

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x000001C8FBF910F0>, 'hidden_units': [10, 20, 30], 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x000001C8F5F7BA60>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

Xem đồ thị của mạng nơ ron bằng cách vào folder hiện tại của commandline và gõ lệnh:

`tensorboard --logdir tmp\iris_model`

In [38]:
acc = classifier.evaluate(input_fn = get_input_test, steps = 1)['accuracy']
print('Accuracy on test: {:.2f} %'.format(acc*100))

INFO:tensorflow:Starting evaluation at 2018-10-23-04:51:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tmp/iris_model\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-10-23-04:51:09
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.96666664, global_step = 2000, loss = 0.0840668
Accuracy on test: 96.67 %


In [46]:
# Predict for new samples
def new_samples():
    return np.array(
        [[6.4, 3.2, 4.5, 1.5],
        [5.8, 3.1, 5.0, 1.7]], dtype = np.float32)

pred = list(classifier.predict(input_fn = new_samples))
print('Sample: %s'%str(new_sample()))
print('Prediction classes: %s'%pred)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tmp/iris_model\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Sample: [[6.4 3.2 4.5 1.5]
 [5.8 3.1 5.  1.7]]
Prediction classes: [1, 2]


# 2. Sử dụng Estimator.

In [40]:
import pandas as pd
pd_train = pd.read_csv(IRIS_TRAINING, header = 0, 
                       names = ['SepLen', 'SepHei', 'PenLen', 'PenHei', 'Class'], 
                       encoding = 'utf-8', engine = 'python')

pd_test = pd.read_csv(IRIS_TEST, header = 0, 
                      names = ['SepLen', 'SepHei', 'PenLen', 'PenHei', 'Class'], 
                      encoding = 'utf-8', engine = 'python')

In [41]:
my_feature_columns = []
for key in pd_train.keys()[:-1]:
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
my_feature_columns

[_NumericColumn(key='SepLen', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepHei', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PenLen', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PenHei', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [42]:
#https://www.tensorflow.org/guide/datasets_for_estimators
def train_input_fn(features, labels, batch_size):    
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()


print(train_input_fn(pd_train.iloc[:, :-1], pd_train.iloc[:, -1], 100))


def test_input_fn(features, labels, batch_size):    
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    # Shuffle, repeat, and batch the examples.
    dataset = dataset.batch(batch_size)
    return dataset


# print(test_input_fn(pd_test.iloc[:, :-1], pd_test.iloc[:, -1], 100))

({'SepLen': <tf.Tensor 'IteratorGetNext_10:3' shape=(?,) dtype=float64>, 'SepHei': <tf.Tensor 'IteratorGetNext_10:2' shape=(?,) dtype=float64>, 'PenLen': <tf.Tensor 'IteratorGetNext_10:1' shape=(?,) dtype=float64>, 'PenHei': <tf.Tensor 'IteratorGetNext_10:0' shape=(?,) dtype=float64>}, <tf.Tensor 'IteratorGetNext_10:4' shape=(?,) dtype=int64>)


In [50]:
#Build a new classifier
# Build 3 layer DNN with 10, 20, 30 units respectively.
classifier2 = tf.contrib.estimator.DNNEstimator(feature_columns = my_feature_columns,
                                           hidden_units = [10, 20, 30],
                                           head = tf.contrib.estimator.multi_class_head(n_classes=3),
                                           optimizer=tf.train.ProximalAdagradOptimizer(
                                                     learning_rate=0.1,
                                                     l1_regularization_strength=0.001
                                           ),
                                           model_dir = 'tmp/iris_model_v2')

#help(tf.contrib.estimator.DNNEstimator)
classifier2.train(input_fn = lambda:train_input_fn(pd_train.iloc[:, :-1], pd_train.iloc[:, -1], 100), steps=2000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tmp/iris_model_v2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F34ADCADD8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into tmp/iris_model_v2\model.ckpt.
IN

In [51]:
acc = classifier2.evaluate(input_fn = test_input_fn, steps = 1)['accuracy']
print('Accuracy on test: {:.2f} %'.format(acc*100))

TypeError: test_input_fn() missing 3 required positional arguments: 'features', 'labels', and 'batch_size'

# 3. Sử dụng Estimator

https://www.tensorflow.org/guide/custom_estimators

In [22]:
def my_model(features, labels, mode, params):
    "Xây dựng mạng DNN với 3 layer ẩn, và xác xuất dropout các unit là 0.1"
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units = units, activation = tf.nn.relu)
        
        #Tính giá trị xác xuất cho mỗi class
        logits = tf.layers.dense(net, params['n_classes'], activation = tf.nn.softmax)
        
        #Tính class dự báo.
        predicted_classes = tf.argmax(logits, 1)
        if mode == tf.estimator.ModeKeys.PREDICT:
            predictions = {
                'class_ids': predicted_classes[:, tf.newaxis],
                'probability': tf.nn.softmax(logits),
                'logits':logits
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)
        
        #Tính loss.
        loss = tf.losses.sparse_softmax_cross_entropy(labels = labels, logits = logits)
        
        #Đánh giá mô hình.
        accuracy = tf.metrics.accuracy(labels = labels,
                                      predictions = predicted_classes,
                                      name = 'acc_op')
        
        metrics = {'accuracy': accuracy}
        tf.summary.scalar('accuracy', accuracy[1])
        
        
        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(
                mode, loss = loss, eval_metric_ops = metrics
            )
        
        #Tạo training operation
        assert mode == tf.estimator.ModeKeys.TRAIN
        optimizer = tf.train.AdagradOptimizer(learning_rate = 0.1)
        train_op = optimizer.minimize(loss, global_step = tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode, loss = loss, train_op = train_op)

In [9]:
import pandas as pd
pd_train = pd.read_csv(IRIS_TRAINING, header = 0, 
                       names = ['SepLen', 'SepHei', 'PenLen', 'PenHei', 'Class'], 
                       encoding = 'utf-8', engine = 'python')

pd_test = pd.read_csv(IRIS_TEST, header = 0, 
                      names = ['SepLen', 'SepHei', 'PenLen', 'PenHei', 'Class'], 
                      encoding = 'utf-8', engine = 'python')

In [4]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

In [5]:
train_X, train_y = pd_train, pd_train.pop('Class')
test_X, test_y = pd_test, pd_test.pop('Class')

In [18]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_X.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
my_feature_columns

[_NumericColumn(key='SepLen', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepHei', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PenLen', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PenHei', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [23]:
# Xây dựng 3 hidden layer với số units lần lượt là 10, 20, 30 ở mỗi layer.
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    params={
        'feature_columns': my_feature_columns,
        'hidden_units': [10, 20, 30],
        'n_classes': 3,
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LAPTOP~1\\AppData\\Local\\Temp\\tmpko5k1sde', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F344F08A20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
# train_input_fn(train_X, train_y, batch_size = 100)
train_input_fn(pd_train.iloc[:, :-1], pd_train.iloc[:, -1], batch_size = 100)

({'PenHei': <tf.Tensor 'IteratorGetNext_4:0' shape=(?,) dtype=float64>,
  'PenLen': <tf.Tensor 'IteratorGetNext_4:1' shape=(?,) dtype=float64>,
  'SepHei': <tf.Tensor 'IteratorGetNext_4:2' shape=(?,) dtype=float64>,
  'SepLen': <tf.Tensor 'IteratorGetNext_4:3' shape=(?,) dtype=float64>},
 <tf.Tensor 'IteratorGetNext_4:4' shape=(?,) dtype=int64>)

In [24]:
batch_size = 100
train_steps = 1000
# Train mô hình
classifier.train(
    input_fn=lambda:train_input_fn(train_X, train_y, batch_size),
    steps=train_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\LAPTOP~1\AppData\Local\Temp\tmpko5k1sde\model.ckpt.
INFO:tensorflow:loss = 1.1779083, step = 1
INFO:tensorflow:global_step/sec: 229.836
INFO:tensorflow:loss = 0.6757853, step = 101 (0.438 sec)
INFO:tensorflow:global_step/sec: 320.806
INFO:tensorflow:loss = 0.6312725, step = 201 (0.313 sec)
INFO:tensorflow:global_step/sec: 314.763
INFO:tensorflow:loss = 0.60638124, step = 301 (0.318 sec)
INFO:tensorflow:global_step/sec: 314.74
INFO:tensorflow:loss = 0.5983583, step = 401 (0.317 sec)
INFO:tensorflow:global_step/sec: 330.097
INFO:tensorflow:loss = 0.5777912, step = 501 (0.305 sec)
INFO:tensorflow:global_step/sec: 304.291
INFO:tensorflow:loss = 0.5833523, step = 601 (0.327 sec)
INFO:tensorflow:global_st

In [25]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [139]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_X, test_y, batch_size))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-11:51:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\LAPTOP~1\AppData\Local\Temp\tmpdq049w0d\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-11:51:26
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.93333334, global_step = 1000, loss = 0.06848378
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\LAPTOP~1\AppData\Local\Temp\tmpdq049w0d\model.ckpt-1000


In [143]:
# Predict model
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepLen': [5.1, 5.9, 6.9],
    'SepHei': [3.3, 3.0, 3.1],
    'PenLen': [1.7, 4.2, 5.4],
    'PenHei': [0.5, 1.5, 2.1],
}

In [144]:
predictions = classifier.predict(
input_fn=lambda:eval_input_fn(predict_x,
                                        labels=None,
                                        batch_size=batch_size))

In [148]:
for pred_dict, expec in zip(predictions, expected):
        template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(class_id,
                              100 * probability, expec))